<a href="https://colab.research.google.com/github/trchudley/GEOG2462/blob/main/Short_Scripts/Week_2_Calculate_NDIs_Median_Landsat_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Calculate a median-composited normalised difference index _for Landsat 5_ in Google Earth Engine

This script calcualtes the _median_ pixel value of all images in the imageCollection, rather than just taking the first image. This could be useful if you are examining changes over larger areas and/or longer timescales.

This script provides an additional opportunity to use Landsat 5, **at your own risk** (it should be perfectly possible to succeed in this assessment using the 10 years of Landsat 8 you have access to!). Landsat 5 was launched in 1984, and the TM sensor failed in 2012. As Landsat 6 did not make orbit, and Landsat 7 developed a scan line corrector issue which made data difficult to use, Landsat 5 was kept going for so long that it holds the title of the longest-operating Earth-observing satellite mission in history.

The Thematic Mapper (TM) sensing instrument on board Landsat 5 is not the same Operational Land Imager (OLI) on board Landsat 8. As a result **bands have different radiometric resolutions and number designations**. Note that below, I have calculated NDSI using Bands 2 and 5 rather than 3 and 6. This is because, in the TM, the analogous (but not entirely identical) green band is 2 and SWIR band is 5.

> ⚠️
> **To convert your own NDI equations to the appropriate bands, you will have to consult the following tables for the TM and OLI equivalent bands**:
> https://www.usgs.gov/faqs/what-are-band-designations-landsat-satellites




## Log in to Google Earth Engine

In [ ]:
import ee
import geemap
import geemap.chart as chart
import time

ee.Authenticate()  # Trigger the authentication flow.
ee.Initialize(project='ee-trchudley')    # Change to your own default project name.

## Set editable parameters

Most editable parameters are in this cell. Thresholding parameters must also be manually selected after the histogram is generated.

In [ ]:

# Location - editable
latitude = -13.922           # Degrees of latitude
longitude = -70.821          # Degrees of longitude
size = 30000                 # Size of AOI, in metres
location_name = 'quelcayya'  # recognisable name, to create a useful file name

# Dates - editable
date_start = '1990-07-15'
date_end = '1990-09-1'

# Set NDI name and r_high and r_low bands
type_of_ndi = 'NDSI'  # type of NDI calculated, for filename purposes
r_high = 'B2'  # relevant band name for r_high
r_low = 'B5'   # relevant band name for r_low

# Google Drive export folder
folder = 'scires_project_2A'

## Search for image and calculate NDI

In [ ]:
# Set up location geometry
point = ee.Geometry.Point(longitude, latitude)  # Create a point
region = point.buffer(size/2).bounds()  # Buffer the point to a 2D shape

# Get Landsat 8 image collection
landsat5_collection = ee.ImageCollection("LANDSAT/LT05/C02/T1_TOA")

# Filter to desired region and date bounds
landsat5_collection = landsat5_collection.filterBounds(region)
landsat5_collection = landsat5_collection.filterDate(date_start, date_end)

# Get the least cloudy image in the collection, and clip it to our search region
image = landsat5_collection.median()
# image = landsat5_collection.sort('CLOUD_COVER').first()
image = image.clip(region)

# Print out date, for reference
# date = image.get('DATE_ACQUIRED').getInfo()
list_length = len(landsat5_collection.toList(landsat5_collection.size()).getInfo())
print(f'Produced composite from {list_length} images')

# Calculate NDI
ndi = image.normalizedDifference([r_high, r_low]).rename(type_of_ndi)


Produced composite from 4 images


Visualise NDI for quality assessment.

In [ ]:

Map = geemap.Map() # Create empty map

# Display colour image
max_reflectance = 0.80
visParams = {'bands': ['B3', 'B2', 'B1'], 'max': max_reflectance}
Map.addLayer(image, visParams, 'True Colour')

# Display NDI
visParams = {'bands': [type_of_ndi], 'min': -1, 'max': 1, 'palette': ['red', 'white', 'blue']}
Map.addLayer(ndi, visParams, type_of_ndi)

Map.centerObject(region, zoom=12)
Map


Map(center=[-13.921960750262597, -70.82079023084508], controls=(WidgetControl(options=['position', 'transparen…

# Thresholding

Use the below histogram to assess viable threshold.

> NOTE: This tool can be quite finicky for median data, and will likely not work. This isn't too much of a problem - you can skip to the next cells and try and iteratively determine a good threshold manually.

In [ ]:

# Sample 10,000 pixels within the NDI image
sample_pixels = ndi.sample(region, numPixels=10000)

# Set labels for the graph
labels = {
    "title": 'Distribution of NDI values within image',
    "xlabel": f'{type_of_ndi} values',
    "ylabel": 'Pixel count',
}

# Construct the histogram
chart.feature_histogram(sample_pixels, type_of_ndi, **labels)

EEException: Element.propertyNames: Parameter 'element' is required and may not be null.

Manually set threshold filters below. Note the `direction` variable, which can be set to `'greater_than'` or `'less_than`' depending on whether you want to classify surfaces > or < the NDI threshold.

The `filter_threshold_km2` sets a minimum size of classified area to include. It can be set to zero if you don't want to filter.

In [ ]:
# manually set an NDI threshold with which to classify
threshold = 0.8

# Greater than or less then
direction = 'greater_than'
# direction = 'less_than'

# Set the minimum size of thresholded areas to be included, in km2. Set to zero if you don't want to filter.
filter_threshold_km2 = 0.5

The rest is automatic:

In [ ]:
# threshold the image to where greater than or less than threshold.
if direction == 'greater_than':
    ndi_threshold = ndi.gte(threshold)
elif direction == 'less_than':
    ndi_threshold = ndi.lte(threshold)
else:
    raise ValueError("`direction` variable must be set to 'greater_than' or 'less_than'")

# 'Mask' the data, showing only regions beyond the threshold.
ndi_threshold = ndi_threshold.updateMask(ndi_threshold.neq(0))

# Calculate the area of each pixel in km2, and add it as a band to the NDI image
ndi_and_area = ndi_threshold.addBands(ndi_threshold.multiply(30*30).divide(1e6))

# Use the `reduceToVectors()` function to produce vectors, also calculating
# the total area using the `sum()` function to sum the pixel areas.
vectors = ndi_and_area.reduceToVectors(
  scale=30,
  geometryType = 'polygon',
  eightConnected = False,
  reducer = ee.Reducer.sum(),
  maxPixels=1e9,
)

# Extra line to rename the area column to `area_km2`.
vectors = vectors.map(lambda feature: feature.set('area_km2', feature.get('sum')).set('sum', None).set('label', None))

# Filter polygons smaller than this chosen threshold
vectors_filtered = vectors.filter(ee.Filter.gte('area_km2', filter_threshold_km2))


Visualise threshold for quality assessment. The unfiltered thresholded region is included as a cyan raster; the area-filtered region is included as a blue vector.

In [ ]:

Map = geemap.Map() # Create empty map

# Display colour image
max_reflectance = 0.80
visParams = {'bands': ['B3', 'B2', 'B1'], 'max': max_reflectance}
Map.addLayer(image, visParams, 'True Colour')

# Display thresholded NDI as raster
visParams = {'bands': [type_of_ndi], 'palette': ['white', 'cyan'], 'opacity': 0.3}
Map.addLayer(ndi_threshold, visParams, type_of_ndi)

# Display vectors filtered
Map.addLayer(vectors_filtered, {'color': 'blue'}, "Identified Region")  # Add our AOI

Map.centerObject(region, zoom=12)
Map


Map(center=[-13.921960750262597, -70.82079023084508], controls=(WidgetControl(options=['position', 'transparen…

Print quantiative statistics:

In [ ]:

# Get total area of initial search region
aoi_area_km2 = region.area(maxError=1).getInfo() / 1e6

# Get total area of all vectors
vector_area_km2 = vectors.aggregate_sum('area_km2').getInfo()

# Get total area the filtered vectors
vector_flt_area_km2 = vectors_filtered.aggregate_sum('area_km2').getInfo()

# Print the results
print(f'Total scene area: {aoi_area_km2:.2f} km2')
print(f'Total classified area: {vector_area_km2:.2f} km2')
print(f'Total classified area (filtered): {vector_flt_area_km2:.2f} km2')


## Download data

Download initial scene:

In [ ]:
# Construct the filename automatically
date_string = image.get('DATE_ACQUIRED').getInfo()
filename = location_name + '_' + date_start + '_' date_end + '_median_' + type_of_ndi

# Print out filename for reference
print("The image will be saved to your Google Drive at:\n" + folder + '/' + filename + '.tif\n')

# Export the image, specifying scale and region.
task = ee.batch.Export.image.toDrive(**{
    'image': image.select(['B3', 'B2', 'B1', 'B4', 'B5']),
    'description': filename,
    'folder': folder,
    'scale': 30,
    'region': region.getInfo()['coordinates']
})
task.start()

while task.active():
  print('Task processing ongoing... (id: {}).'.format(task.id))
  time.sleep(5)

print('Finished processing. Image is exported to your Drive.')


Download NDI:

In [ ]:
# Construct the filename automatically
filename = location_name + '_' + date_start + '_' + date_end + '_median_' + type_of_ndi

# Print out filename for reference
print("The image will be saved to your Google Drive at:\n" + folder + '/' + filename + '.tif\n')

# Export the image, specifying scale and region.
task = ee.batch.Export.image.toDrive(**{
    'image': ndi,
    'description': filename,
    'folder': folder,
    'scale': 30,
    'region': region.getInfo()['coordinates']
})
task.start()

while task.active():
  print('Task processing ongoing... (id: {}).'.format(task.id))
  time.sleep(5)

print('Finished processing. Image is exported to your Drive.')


Download vector data:

In [ ]:
# Construct the filename automatically
filename = location_name + '_' + date_start + '_' + date_end + '_' + type_of_ndi + '_theshold_' + str(max_reflectance)

# Print out filename for reference
print("The image will be saved to your Google Drive at:\n" + folder + '/' + filename + '.kml\n')

# Export the featureCollection, specifying scale and region.
task = ee.batch.Export.table.toDrive(**{
  'collection': vectors_filtered,
  'description': filename,
  'folder': folder,
  # 'fileFormat': 'KML',
  'fileFormat': 'SHP',
})
task.start()

while task.active():
  print('Task processing ongoing... (id: {}).'.format(task.id))
  time.sleep(5)

print('Finished processing. Vector file is exported to your Drive.')
